## Purpose:

According to and article from [brookfieldresidential.com](https://stories.brookfieldresidential.com/homebuyersschool/duplex-vs.-single-family-home-whats-the-difference-and-which-one-should-i-invest-in), duplexes might be more highly valued than single family homes. Using the Seattle area real estate information, I'll run some analysis to determine the accuracy of this claim.

In [1]:
#add auto reload for src function testing
%load_ext autoreload
%autoreload 2

#let's add the project directory to our module path
import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
    
#also import all of our modules
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from src import data_cleaning

#and here is our data directiory
data_folder = '../../data/'

Let's open our data and see what it looks like. I'm looking for both housing prices and whether or not they are duplexes.

## Optional

The following lines of code will take a while to load. This is because the csv files are very large and contain data that we don't need. We solved this by creating new csv files that contain data for only 2019. Read through the following section to see how we did that, and uncomment the code if you want to follow along. Otherwise skip ahead to 2019 data import header.

### EXTR_RPSale.csv

It looks like this data set is what contains the sale price as well as some interesting characteristics of the property like whether it's historic or not. 

In [2]:
# rp_sale = pd.read_csv(data_folder+'EXTR_RPSale.csv')
# rp_sale.info()
# rp_sale.head()

### Date
I know I only want to look at records from 2019 so I'm going to filter out the data to only include entries from that date.

First I'll convert document date to datetime:

In [3]:
# rp_sale['DocumentDate'] = pd.to_datetime(rp_sale['DocumentDate'])

# rp_sale.info()

Now I'm going to create a function so that I can make a dataframe mask using apply. It will check whether the year attribute of a datetime object equals 2019 and, if so, it will return True, otherwise False.

In [4]:
# def in_2019(dateTime):
#     if dateTime.year == 2019:
#         return True
#     else:
#         return False

In [5]:
# mask_2019 = rp_sale['DocumentDate'].apply(in_2019)


# rp_sale_2019 = rp_sale[mask_2019]
# rp_sale_2019.info()
# rp_sale_2019.head(20)

I want to use this dataframe for all my analysis, so I'm going to export it as a csv, and I'll create a function to automatically do this as well.

In [6]:
#uncomment line below to create file

#rp_sale_2019.to_csv(data_folder+'EXTR_RPSale_2019.csv')

### 2019 data import function testing:

In [7]:
# rp_sale_2019 = data_cleaning.filter_data_by_year(rp_sale)

In [8]:
# rp_sale_2019.info()

In [9]:
# data_cleaning.create_2019_sale_csv(rp_sale)

# 2019 sales data import

In [44]:
rp_sale = pd.read_csv(data_folder+'EXTR_RPSale_2019.csv')

In [45]:
rp_sale.info()
rp_sale.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61351 entries, 0 to 61350
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          61351 non-null  int64 
 1   ExciseTaxNbr        61351 non-null  int64 
 2   Major               61351 non-null  int64 
 3   Minor               61351 non-null  int64 
 4   DocumentDate        61351 non-null  object
 5   SalePrice           61351 non-null  int64 
 6   RecordingNbr        61351 non-null  object
 7   Volume              61351 non-null  object
 8   Page                61351 non-null  object
 9   PlatNbr             61351 non-null  object
 10  PlatType            61351 non-null  object
 11  PlatLot             61351 non-null  object
 12  PlatBlock           61351 non-null  object
 13  SellerName          61351 non-null  object
 14  BuyerName           61351 non-null  object
 15  PropertyType        61351 non-null  int64 
 16  PrincipalUse        61

,Unnamed: 0,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,72,2999169,919715,200,2019-07-08,192000,20190712001080,,,,...,3,2,3,N,N,N,N,1,3,
1,236,3000673,894444,200,2019-06-26,185000,20190722001395,,,,...,3,2,3,N,N,N,N,1,3,
2,257,3027422,213043,120,2019-12-20,560000,20191226000848,,,,...,11,6,3,N,N,N,N,1,8,
3,302,3002257,940652,630,2019-07-22,435000,20190730001339,,,,...,11,6,3,N,N,N,N,1,8,
4,446,3018109,152504,9008,2019-10-18,7600000,20191030001615,,,,...,3,7,3,N,N,N,N,1,2,
5,465,2993601,140281,20,2019-06-04,450000,20190614000489,,,,...,3,6,3,N,N,N,N,1,8,
6,482,3015516,779790,30,2019-10-07,0,20191016000009,,,,...,11,6,3,N,N,N,N,1,8,
7,586,3031504,766620,3538,2019-12-30,0,20200128000956,,,,...,51,7,15,N,N,N,N,18,2,
8,594,3015264,124550,98,2019-09-27,193000,20191015000395,,,,...,3,6,15,N,N,N,N,18,8,18 51 52
9,599,2980648,797320,2320,2019-03-27,540000,,,,,...,3,6,3,N,N,N,N,1,8,


## Creating the PIN column

The unique identifier for each piece of land is the PIN, which is made up of the Major and Minor columns. I'll turn the major and minor rows into a pin row so that I can easily join the  First I'll change everything into a string, I'll zero pad them, then I'll join them.

In [46]:
rp_sale['Major'] = rp_sale['Major'].apply(str)
rp_sale['Minor'] = rp_sale['Minor'].apply(str)

In [47]:
rp_sale.info()
rp_sale.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61351 entries, 0 to 61350
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          61351 non-null  int64 
 1   ExciseTaxNbr        61351 non-null  int64 
 2   Major               61351 non-null  object
 3   Minor               61351 non-null  object
 4   DocumentDate        61351 non-null  object
 5   SalePrice           61351 non-null  int64 
 6   RecordingNbr        61351 non-null  object
 7   Volume              61351 non-null  object
 8   Page                61351 non-null  object
 9   PlatNbr             61351 non-null  object
 10  PlatType            61351 non-null  object
 11  PlatLot             61351 non-null  object
 12  PlatBlock           61351 non-null  object
 13  SellerName          61351 non-null  object
 14  BuyerName           61351 non-null  object
 15  PropertyType        61351 non-null  int64 
 16  PrincipalUse        61

,Unnamed: 0,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,72,2999169,919715,200,2019-07-08,192000,20190712001080,,,,...,3,2,3,N,N,N,N,1,3,
1,236,3000673,894444,200,2019-06-26,185000,20190722001395,,,,...,3,2,3,N,N,N,N,1,3,
2,257,3027422,213043,120,2019-12-20,560000,20191226000848,,,,...,11,6,3,N,N,N,N,1,8,
3,302,3002257,940652,630,2019-07-22,435000,20190730001339,,,,...,11,6,3,N,N,N,N,1,8,
4,446,3018109,152504,9008,2019-10-18,7600000,20191030001615,,,,...,3,7,3,N,N,N,N,1,2,
5,465,2993601,140281,20,2019-06-04,450000,20190614000489,,,,...,3,6,3,N,N,N,N,1,8,
6,482,3015516,779790,30,2019-10-07,0,20191016000009,,,,...,11,6,3,N,N,N,N,1,8,
7,586,3031504,766620,3538,2019-12-30,0,20200128000956,,,,...,51,7,15,N,N,N,N,18,2,
8,594,3015264,124550,98,2019-09-27,193000,20191015000395,,,,...,3,6,15,N,N,N,N,18,8,18 51 52
9,599,2980648,797320,2320,2019-03-27,540000,,,,,...,3,6,3,N,N,N,N,1,8,


In [48]:
# pad the Major and minor

rp_sale['Major'] = rp_sale['Major'].apply(lambda elem: elem.rjust(6, '0'))
rp_sale['Minor'] = rp_sale['Minor'].apply(lambda elem: elem.rjust(4, '0'))

rp_sale.head(10)

,Unnamed: 0,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,72,2999169,919715,0200,2019-07-08,192000,20190712001080,,,,...,3,2,3,N,N,N,N,1,3,
1,236,3000673,894444,0200,2019-06-26,185000,20190722001395,,,,...,3,2,3,N,N,N,N,1,3,
2,257,3027422,213043,0120,2019-12-20,560000,20191226000848,,,,...,11,6,3,N,N,N,N,1,8,
3,302,3002257,940652,0630,2019-07-22,435000,20190730001339,,,,...,11,6,3,N,N,N,N,1,8,
4,446,3018109,152504,9008,2019-10-18,7600000,20191030001615,,,,...,3,7,3,N,N,N,N,1,2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61346,2087782,2986698,033310,0255,2019-04-30,6500000,20190508000847,,,,...,11,6,3,N,N,N,N,1,8,
61347,2087856,2980997,334840,1022,2019-04-03,29000,20190405000544,,,,...,1,6,18,N,N,N,N,18,7,13 31
61348,2087900,2997920,302300,0320,2019-05-01,0,20190705000539,,,,...,11,6,15,N,N,N,N,1,8,
61349,2087907,3028691,277110,4239,2019-12-19,955000,20200103000685,,,,...,10,6,3,N,N,N,N,1,8,


In [49]:
rp_sale['PIN'] = rp_sale['Major'] + rp_sale['Minor']
rp_sale.head(10)

,Unnamed: 0,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,...,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,PIN
0,72,2999169,919715,0200,2019-07-08,192000,20190712001080,,,,...,2,3,N,N,N,N,1,3,,9197150200
1,236,3000673,894444,0200,2019-06-26,185000,20190722001395,,,,...,2,3,N,N,N,N,1,3,,8944440200
2,257,3027422,213043,0120,2019-12-20,560000,20191226000848,,,,...,6,3,N,N,N,N,1,8,,2130430120
3,302,3002257,940652,0630,2019-07-22,435000,20190730001339,,,,...,6,3,N,N,N,N,1,8,,9406520630
4,446,3018109,152504,9008,2019-10-18,7600000,20191030001615,,,,...,7,3,N,N,N,N,1,2,,1525049008
5,465,2993601,140281,0020,2019-06-04,450000,20190614000489,,,,...,6,3,N,N,N,N,1,8,,1402810020
6,482,3015516,779790,0030,2019-10-07,0,20191016000009,,,,...,6,3,N,N,N,N,1,8,,7797900030
7,586,3031504,766620,3538,2019-12-30,0,20200128000956,,,,...,7,15,N,N,N,N,18,2,,7666203538
8,594,3015264,124550,0098,2019-09-27,193000,20191015000395,,,,...,6,15,N,N,N,N,18,8,18 51 52,1245500098
9,599,2980648,797320,2320,2019-03-27,540000,,,,,...,6,3,N,N,N,N,1,8,,7973202320


In [51]:
rp_sale.drop(axis=1, labels='Unnamed: 0', inplace=True)
rp_sale.head(10)

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,PIN
0,2999169,919715,0200,2019-07-08,192000,20190712001080,,,,,...,2,3,N,N,N,N,1,3,,9197150200
1,3000673,894444,0200,2019-06-26,185000,20190722001395,,,,,...,2,3,N,N,N,N,1,3,,8944440200
2,3027422,213043,0120,2019-12-20,560000,20191226000848,,,,,...,6,3,N,N,N,N,1,8,,2130430120
3,3002257,940652,0630,2019-07-22,435000,20190730001339,,,,,...,6,3,N,N,N,N,1,8,,9406520630
4,3018109,152504,9008,2019-10-18,7600000,20191030001615,,,,,...,7,3,N,N,N,N,1,2,,1525049008
5,2993601,140281,0020,2019-06-04,450000,20190614000489,,,,,...,6,3,N,N,N,N,1,8,,1402810020
6,3015516,779790,0030,2019-10-07,0,20191016000009,,,,,...,6,3,N,N,N,N,1,8,,7797900030
7,3031504,766620,3538,2019-12-30,0,20200128000956,,,,,...,7,15,N,N,N,N,18,2,,7666203538
8,3015264,124550,0098,2019-09-27,193000,20191015000395,,,,,...,6,15,N,N,N,N,18,8,18 51 52,1245500098
9,2980648,797320,2320,2019-03-27,540000,,,,,,...,6,3,N,N,N,N,1,8,,7973202320


## Joining rp_sales with Res building info

Now that I have the Parcel ID number in my sales info, I want to join it with my resbldg number. Let's take a look at that dataframe.

In [52]:
res_bldg = pd.read_csv(data_folder+'EXTR_ResBldg.csv')

/home/max/anaconda3/envs/housing/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [53]:
res_bldg.info()
res_bldg.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515147 entries, 0 to 515146
Data columns (total 50 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Major               515147 non-null  int64  
 1   Minor               515147 non-null  int64  
 2   BldgNbr             515147 non-null  int64  
 3   NbrLivingUnits      515147 non-null  int64  
 4   Address             515147 non-null  object 
 5   BuildingNumber      515147 non-null  object 
 6   Fraction            515147 non-null  object 
 7   DirectionPrefix     514643 non-null  object 
 8   StreetName          515147 non-null  object 
 9   StreetType          515147 non-null  object 
 10  DirectionSuffix     514643 non-null  object 
 11  ZipCode             469753 non-null  object 
 12  Stories             515147 non-null  float64
 13  BldgGrade           515147 non-null  int64  
 14  BldgGradeVar        515147 non-null  int64  
 15  SqFt1stFloor        515147 non-nul

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,9800,440,1,1,2053 277TH AVE SE 98075,2053,,,277TH,AVE,...,0,0,0,2004,0,0,0,0,3,0
1,9800,710,1,1,27713 SE 26TH WAY 98075,27713,,SE,26TH,WAY,...,0,0,0,2002,0,0,0,0,3,0
2,9800,720,1,1,27719 SE 26TH WAY 98075,27719,,SE,26TH,WAY,...,0,0,0,2001,0,0,0,0,3,0
3,9802,30,1,1,2831 278TH AVE SE 98075,2831,,,278TH,AVE,...,0,0,0,2004,0,0,0,0,3,0
4,9802,140,1,1,2829 277TH TER SE 98075,2829,,,277TH,TER,...,0,0,0,2004,0,0,0,0,3,0


Let's make a function from the last time I made the pin column and use it on this dataframe too.

In [54]:
def add_PIN_column(df):
    """
    input: dataframe with Major and Minor columns
    output: dataframe with PIN column added
    """
    # turn the major and minor columns into strings
    df['Major'] = df['Major'].apply(str)
    df['Minor'] = df['Minor'].apply(str)
    
    # pad each column with zeros
    df['Major'] = df['Major'].apply(lambda elem: elem.rjust(6, '0'))
    df['Minor'] = df['Minor'].apply(lambda elem: elem.rjust(4, '0'))
    
    #create pin column
    df['PIN'] = df['Major'] + df['Minor']
    
    return df

Now let's test it out and try joining the two dataframes (sales and building)

In [55]:
res_bldg = add_PIN_column(res_bldg)
res_bldg.head()


,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost,PIN
0,009800,0440,1,1,2053 277TH AVE SE 98075,2053,,,277TH,AVE,...,0,0,2004,0,0,0,0,3,0,0098000440
1,009800,0710,1,1,27713 SE 26TH WAY 98075,27713,,SE,26TH,WAY,...,0,0,2002,0,0,0,0,3,0,0098000710
2,009800,0720,1,1,27719 SE 26TH WAY 98075,27719,,SE,26TH,WAY,...,0,0,2001,0,0,0,0,3,0,0098000720
3,009802,0030,1,1,2831 278TH AVE SE 98075,2831,,,278TH,AVE,...,0,0,2004,0,0,0,0,3,0,0098020030
4,009802,0140,1,1,2829 277TH TER SE 98075,2829,,,277TH,TER,...,0,0,2004,0,0,0,0,3,0,0098020140


In [60]:
res_bldg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515147 entries, 0 to 515146
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Major               515147 non-null  object 
 1   Minor               515147 non-null  object 
 2   BldgNbr             515147 non-null  int64  
 3   NbrLivingUnits      515147 non-null  int64  
 4   Address             515147 non-null  object 
 5   BuildingNumber      515147 non-null  object 
 6   Fraction            515147 non-null  object 
 7   DirectionPrefix     514643 non-null  object 
 8   StreetName          515147 non-null  object 
 9   StreetType          515147 non-null  object 
 10  DirectionSuffix     514643 non-null  object 
 11  ZipCode             469753 non-null  object 
 12  Stories             515147 non-null  float64
 13  BldgGrade           515147 non-null  int64  
 14  BldgGradeVar        515147 non-null  int64  
 15  SqFt1stFloor        515147 non-nul

In [68]:
rejects = res_bldg.join(rp_sale.set_index('PIN'), on='PIN', how='outer', lsuffix='_res')

In [71]:
rejects.loc[:,['Major', 'Major_res', 'Minor', 'Minor_res', 'PIN']]

,Major,Major_res,Minor,Minor_res,PIN
0.0,NaN,009800,NaN,0440,0098000440
1.0,NaN,009800,NaN,0710,0098000710
2.0,NaN,009800,NaN,0720,0098000720
3.0,NaN,009802,NaN,0030,0098020030
4.0,NaN,009802,NaN,0140,0098020140
...,...,...,...,...,...
NaN,785345,NaN,0040,NaN,7853450040
NaN,926660,NaN,0940,NaN,9266600940
NaN,572750,NaN,0099,NaN,5727500099
NaN,524780,NaN,0390,NaN,5247800390


In [72]:
rp_sale[rp_sale['PIN'] == '9266600940']

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,PIN
61317,2979656,926660,0940,2019-03-06,199000,20190328001083,,,,,...,2,3,N,N,N,N,1,3,,9266600940


In [74]:
joined = rp_sale.join(res_bldg.set_index('PIN'), on='PIN', how='inner', rsuffix='_res')

In [76]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43838 entries, 2 to 61349
Data columns (total 75 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ExciseTaxNbr        43838 non-null  int64  
 1   Major               43838 non-null  object 
 2   Minor               43838 non-null  object 
 3   DocumentDate        43838 non-null  object 
 4   SalePrice           43838 non-null  int64  
 5   RecordingNbr        43838 non-null  object 
 6   Volume              43838 non-null  object 
 7   Page                43838 non-null  object 
 8   PlatNbr             43838 non-null  object 
 9   PlatType            43838 non-null  object 
 10  PlatLot             43838 non-null  object 
 11  PlatBlock           43838 non-null  object 
 12  SellerName          43838 non-null  object 
 13  BuyerName           43838 non-null  object 
 14  PropertyType        43838 non-null  int64  
 15  PrincipalUse        43838 non-null  int64  
 16  Sale

In [77]:
joined.PrincipalUse.unique()

array([ 6,  7,  2,  3,  9,  0, 11])

In [79]:
rp_sale.PIN.value_counts()

3421059008    20
6061250080    15
8847500004    15
5396650004    14
5101400770    13
              ..
2331400240     1
1858600006     1
1122029082     1
6054710110     1
7006200430     1
Name: PIN, Length: 54232, dtype: int64

In [81]:
joined.PIN.value_counts()

0114100979    77
0822119001    63
2024069115    30
1105000555    16
7548300601    12
              ..
6372000296     1
0724059101     1
7151700310     1
5126300940     1
4037200700     1
Name: PIN, Length: 38351, dtype: int64

In [82]:
res_bldg.PIN.value_counts()

1522049091    31
3223059036    28
0822119001    21
2722059164    19
2422079026    16
              ..
7708200420     1
7201900200     1
3905050510     1
7701500030     1
1837010120     1
Name: PIN, Length: 509573, dtype: int64

In [87]:
res_bldg[res_bldg['PIN']=='1522049091']

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost,PIN
143015,152204,9091,29,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,1963,0,0,0,0,4,0,1522049091
143136,152204,9091,11,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,1963,0,0,0,0,4,0,1522049091
143137,152204,9091,9,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,1963,0,0,0,0,4,0,1522049091
143138,152204,9091,1,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,1963,0,0,0,0,4,0,1522049091
143139,152204,9091,19,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,1963,0,0,0,0,4,0,1522049091
143140,152204,9091,20,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,1963,0,0,0,0,4,0,1522049091
143141,152204,9091,6,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,1963,0,0,0,0,4,0,1522049091
143142,152204,9091,8,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,2004,0,0,0,0,4,0,1522049091
143143,152204,9091,22,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,1963,0,0,0,0,4,0,1522049091
143144,152204,9091,28,1,23948 35TH PL S 98032,23948,,,35TH,PL,...,0,0,1963,0,0,0,0,4,0,1522049091


In [88]:
res_bldg[res_bldg['PIN']=='1522049091'].BldgNbr.value_counts()

31    1
15    1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
16    1
30    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
1     1
Name: BldgNbr, dtype: int64

In [90]:
multi = joined[joined['PIN']=='0114100979']

In [95]:
multi.loc[:, ['Address', 'BldgNbr', 'SalePrice']]

,Address,BldgNbr,SalePrice
2935,18254 73RD AVE NE,2,720000
2935,18254 73RD AVE NE,1,720000
2935,18254 73RD AVE NE,10,720000
2935,18254 73RD AVE NE,9,720000
2935,18254 73RD AVE NE,5,720000
...,...,...,...
54530,18254 73RD AVE NE,8,682500
54530,18254 73RD AVE NE,4,682500
54530,18254 73RD AVE NE,3,682500
54530,18254 73RD AVE NE,7,682500


In [92]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43838 entries, 2 to 61349
Data columns (total 75 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ExciseTaxNbr        43838 non-null  int64  
 1   Major               43838 non-null  object 
 2   Minor               43838 non-null  object 
 3   DocumentDate        43838 non-null  object 
 4   SalePrice           43838 non-null  int64  
 5   RecordingNbr        43838 non-null  object 
 6   Volume              43838 non-null  object 
 7   Page                43838 non-null  object 
 8   PlatNbr             43838 non-null  object 
 9   PlatType            43838 non-null  object 
 10  PlatLot             43838 non-null  object 
 11  PlatBlock           43838 non-null  object 
 12  SellerName          43838 non-null  object 
 13  BuyerName           43838 non-null  object 
 14  PropertyType        43838 non-null  int64  
 15  PrincipalUse        43838 non-null  int64  
 16  Sale

In [96]:
multi.SalePrice.value_counts()

715000    11
699500    11
550500    11
519500    11
599500    11
682500    11
720000    11
Name: SalePrice, dtype: int64

In [98]:
multi.BldgNbr.value_counts()

12    7
10    7
9     7
8     7
7     7
6     7
5     7
4     7
3     7
2     7
1     7
Name: BldgNbr, dtype: int64

In [99]:
multi.DocumentDate.value_counts()

2019-08-29    66
2019-12-26    11
Name: DocumentDate, dtype: int64

2985982    197
3014061     68
2988913     62
2986226     61
3026666     37
          ... 
2992607      1
2990558      1
2996701      1
2994652      1
3014656      1
Name: ExciseTaxNbr, Length: 57836, dtype: int64